In [70]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tabulate import tabulate
from colorama import Fore, Style
from abc import ABC, abstractmethod
from typing import Callable, List, Optional, Dict, Tuple
from dataclasses import dataclass, field
import random

# Signal Generator

In [3]:
class SignalGenerator(ABC):
    @abstractmethod
    def generate_signals(self, data: pd.DataFrame) -> pd.Series:
        """
        data: OHLCV (and extra) up to t‑1
        returns: Series indexed like data of +1 (long), -1 (short), 0 (flat)
        """

In [4]:
class SimpleMAMomentum(SignalGenerator):
    """Long when price > its 24h SMA, exit when below 24h SMA"""
    def __init__(self, window: int = 24):
        self.window = window

    def generate_signals(self, data: pd.DataFrame) -> pd.Series:
        close = data['close']
        sma = close.rolling(self.window).mean()
        signal = pd.Series(0, index=data.index)
        signal[close > sma] = 1
        signal[close < sma] = -1
        return signal

In [5]:
class EnhancedMACDRSIStrategy(SignalGenerator):
    """Refined MACD+RSI strategy: includes trend filtering and volatility suppression"""
    def __init__(self,
                 macd_fast:int=12, macd_slow:int=26, macd_signal:int=9,
                 rsi_period:int=14, rsi_oversold:float=30, rsi_overbought:float=70,
                 trend_ema:int=100, min_volatility:float=0.005):
        self.fast = macd_fast
        self.slow = macd_slow
        self.signal = macd_signal
        self.rsi_period = rsi_period
        self.rsi_over = rsi_overbought
        self.rsi_under = rsi_oversold
        self.trend_ema = trend_ema
        self.min_vol = min_volatility

    def generate_signals(self, data:pd.DataFrame) -> pd.Series:
        df = data.copy()
        close = df['close']
        # MACD: fast EMA, slow EMA
        ema_fast = close.ewm(span=self.fast, adjust=False).mean()
        ema_slow = close.ewm(span=self.slow, adjust=False).mean()
        macd_line = ema_fast - ema_slow
        macd_signal = macd_line.ewm(span=self.signal, adjust=False).mean()
        # RSI: standard Wilder's RSI
        delta = close.diff()
        up = delta.clip(lower=0)
        down = -delta.clip(upper=0)
        ema_up = up.ewm(alpha=1/self.rsi_period, adjust=False).mean()
        ema_down = down.ewm(alpha=1/self.rsi_period, adjust=False).mean()
        rs = ema_up / ema_down
        rsi = 100 - (100 / (1 + rs))

        sig = pd.Series(0, index=df.index)

        # Volatility filter (relative daily range)
        atr = (df['high'] - df['low']) / df['close'].shift(1)
        low_vol = atr < self.min_vol

        # Trend filter (price must be above long-term EMA)
        trend = df['close'] > df['close'].ewm(span=self.trend_ema, adjust=False).mean()

        # bullish MACD crossover + RSI oversold + volatility/trend filter
        buy = (
            (macd_line > macd_signal) &
            (macd_line.shift() <= macd_signal.shift()) &
            (rsi < self.rsi_under) &
            trend &
            (~low_vol)
        )
        sell = (
            (macd_line < macd_signal) &
            (macd_line.shift() >= macd_signal.shift()) &
            (rsi > self.rsi_over)
        )

        sig[buy] = 1
        sig[sell] = -1
        return sig

# Backtesting Framework

In [72]:
tick_size: float = 0.01
    freq_hours: float = 1                  # bar frequency in hours

    @property
    def annual_factor(self) -> float:
        periods = 365 * (24 / self.freq_hours)
        return np.sqrt(periods)

In [74]:
@dataclass
class Position:
    entry_bar: int
    entry_price: float
    qty: float
    side: int
    entry_cost: float
    is_maker: bool
    exit_bar: Optional[int] = None
    exit_price: float = 0.0
    exit_cost: float = 0.0
    pnl: float = 0.0
    slippage_notional: float = 0.0

In [76]:
class CostModel:
    def __init__(self, cfg: BacktestConfig, spread_array: np.ndarray):
        self.cfg = cfg
        self.spread_array = spread_array

    def _select_commission(self, notional: float, is_maker: bool) -> float:
        rate = self.cfg.commission_tiers[0][2]
        for thresh, maker, taker in self.cfg.commission_tiers:
            if notional >= thresh:
                rate = maker if is_maker else taker
        return rate

    def slippage(self, price: float, qty: float, volume: float) -> float:
        """
        Nonlinear slippage: k * (qty/volume)^exp
        """
        rel = min(qty/volume, 1)
        return self.cfg.slippage_k * (rel ** self.cfg.slippage_exp)

    def cost_pct(self, price: float, qty: float, idx: int, volume: float,
                 is_maker: bool=False) -> float:
        """Combine fee, spread and slippage"""
        spread = self.spread_array[idx]
        notional = price * qty
        comm = self._select_commission(notional, is_maker)
        slip = self.slippage(price, qty, volume)
        return comm + spread + slip

In [78]:
class PositionManager:
    def __init__(self, cost_model: CostModel, cfg: BacktestConfig, data: pd.DataFrame):
        self.cm = cost_model
        self.cfg = cfg
        self.data = data
        self.position: Optional[Position] = None
        self.trades: List[Position] = []

    def _apply_latency(self) -> int:
        """Sample latency from normal distribution"""
        delay = int(round(random.gauss(self.cfg.latency_mean, self.cfg.latency_std)))
        return max(delay, 0)

    def on_bar(self, i: int, signal: int, equity: float) -> float:
        o,h,l,vol = self.data.open.iat[i], self.data.high.iat[i], self.data.low.iat[i], self.data.volume.iat[i]
        # Entry
        if self.position is None and signal != 0:
            raw_qty = equity * self.cfg.risk_per_trade / (self.cfg.stop_loss * o)
            qty = min(raw_qty * self.cfg.leverage, equity * self.cfg.leverage / o, vol * self.cfg.max_fill_ratio)
            qty = np.floor(qty / self.cfg.min_qty) * self.cfg.min_qty
            if qty > 0:
                # decide maker vs taker randomly
                is_maker = random.random() < 0.2  # 20% maker order assumed
                delay = self._apply_latency()
                idx = min(i+delay, len(self.data)-1)
                price_fill = self.data.open.iat[idx] * (1 + signal/abs(signal) * self.cm.spread_array[idx])
                cost = self.cm.cost_pct(price_fill, qty, idx, self.data.volume.iat[idx], is_maker)
                pos = Position(idx, price_fill, qty, int(np.sign(signal)), cost, is_maker)
                self.position = pos
            return 0.0
        # Exit/SL/TP/margin
        if self.position:
            pos = self.position
            cur_idx = i
            price_cur = o
            sl_price = pos.entry_price * (1 - pos.side * self.cfg.stop_loss)
            tp_price = pos.entry_price * (1 + pos.side * self.cfg.take_profit)
            hit_sl = (pos.side>0 and l<=sl_price) or (pos.side<0 and h>=sl_price)
            hit_tp = (pos.side>0 and h>=tp_price) or (pos.side<0 and l<=tp_price)
            first = 'sl' if hit_sl and (not hit_tp or self.cfg.sl_first) else 'tp' if hit_tp else None
            can = True  # skip min_hold for realism
            execute = False
            target_price = None
            if first == 'sl': execute=True; target_price=sl_price
            elif first == 'tp': execute=True; target_price=tp_price
            elif signal == 0 or np.sign(signal)!=pos.side:
                execute=True; target_price=price_cur
            margin_notional = pos.qty*pos.entry_price*self.cfg.margin_requirement
            if equity < margin_notional:
                execute=True; target_price=l if pos.side>0 else h
            if execute:
                is_maker = False
                delay=self._apply_latency(); idx=min(cur_idx+delay,len(self.data)-1)
                fill_price = target_price * (1 - pos.side*self.cm.spread_array[idx])
                cost = self.cm.cost_pct(fill_price, pos.qty, idx, self.data.volume.iat[idx], is_maker)
                pnl = pos.side*pos.qty*(fill_price-pos.entry_price)
                pos.exit_bar=idx; pos.exit_price=fill_price; pos.exit_cost=cost; pos.pnl=pnl
                pos.slippage_notional = pos.entry_cost*pos.qty*pos.entry_price + cost*pos.qty*fill_price
                self.trades.append(pos); self.position=None
                return pnl
        return 0.0

In [80]:
class MetricsTracker:
    def __init__(self, cfg: BacktestConfig):
        self.cfg=cfg; self.times=[]; self.equity=[]
    def record(self,t,eq):
        self.times.append(t); self.equity.append(eq)
    def compute(self, trades:List[Position]) -> Dict[str,float]:
        eqs=pd.Series(self.equity,index=self.times)
        rtn=eqs.pct_change().dropna(); ann=self.cfg.annual_factor
        sharpe=rtn.mean()/rtn.std()*ann if rtn.std() else np.nan
        cum=(1+rtn).cumprod(); dd=(cum-cum.cummax())/cum.cummax(); mdd=dd.min()
        wins=[tr.pnl for tr in trades if tr.pnl>0]; losses=[tr.pnl for tr in trades if tr.pnl<0]
        stats={
            'Sharpe':sharpe,'MaxDD':mdd,'TotalTrades':len(trades),
            'MeanSlippage':np.mean([t.slippage_notional for t in trades]) if trades else 0
        }
        print(tabulate([[k,f"{v:.4f}"] for k,v in stats.items()], headers=['Metric','Value'], tablefmt='fancy_grid'))
        return stats

In [82]:
class Backtester:
    def __init__(self,data:pd.DataFrame,strategy:Callable[[pd.DataFrame],pd.Series],cfg:BacktestConfig,order_book:Optional[pd.DataFrame]=None):
        idx=pd.date_range(data.index.min(),data.index.max(),freq=f'{cfg.freq_hours}h')
        df=data.reindex(idx).ffill().dropna(subset=['open','high','low','close','volume'])
        spreads=(order_book.reindex(idx).ffill().assign(
            spread=lambda x: (x.ask-x.bid)/(2*df.open)
        ).spread.values if order_book is not None else np.full(len(df),cfg.spread))
        self.data,self.strategy,self.cfg= df,strategy,cfg
        self.cm=CostModel(cfg,spreads)
        self.pm=PositionManager(self.cm,cfg,df)
        self.mt=MetricsTracker(cfg)
    def run(self):
        cash=self.cfg.start_equity; self.mt.record(self.data.index[0],cash)
        sig=self.strategy(self.data).shift(1).fillna(0).astype(int)
        for i in range(1,len(self.data)):
            eq_before=cash+(self.pm.position.side*self.pm.position.qty*(self.data.close.iat[i]-self.pm.position.entry_price) if self.pm.position else 0)
            pnl=self.pm.on_bar(i,int(sig.iat[i]),eq_before); cash+=pnl
            self.mt.record(self.data.index[i],cash)
        stats=self.mt.compute(self.pm.trades)
        # simplified plots
        fig,axs=plt.subplots(2,2,figsize=(12,8))
        eqs=pd.Series(self.mt.equity,index=self.mt.times)
        axs[0,0].plot(eqs); axs[0,0].set_title('Equity')
        axs[0,1].plot((1+eqs.pct_change().fillna(0)).cumprod()); axs[0,1].set_title('Cumulative')
        axs[1,0].hist(eqs.pct_change().dropna(),bins=30); axs[1,0].set_title('Returns Dist')
        axs[1,1].plot(dd.index,dd.values); axs[1,1].set_title('Drawdown')
        plt.tight_layout(); plt.show()
        return stats

In [84]:
import pandas as pd

# Load without parsing dates
df = pd.read_csv("ohlcv.csv")

# Convert UNIX timestamp (ms) to datetime
df["start_time"] = pd.to_datetime(df["start_time"], unit="ms")

# Set index to datetime
df.set_index("start_time", inplace=True)

# Sort index for consistency
df = df.sort_index()

# Make column names lowercase
df.columns = df.columns.str.lower()

# Now run the backtest
cfg = BacktestConfig()

# Simple SMA momentum
sma_strat = SimpleMAMomentum(window=24)
bt1 = Backtester(df, sma_strat.generate_signals, cfg)
res1 = bt1.run()

# Enhanced MACD+RSI strategy
macd_strat = EnhancedMACDRSIStrategy()
bt2 = Backtester(df, macd_strat.generate_signals, cfg)
res2 = bt2.run()


AttributeError: 'BacktestConfig' object has no attribute 'spread'

In [179]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tabulate import tabulate
from dataclasses import dataclass, field
from typing import Callable, List, Optional, Dict, Tuple
import random

@dataclass
class BacktestConfig:
    """
    Backtest configuration: risk, execution, data handling parameters.
    """
    start_equity: float = 10_000.0
    risk_per_trade: float = 0.01
    leverage: float = 1.0
    stop_loss: float = 0.02
    take_profit: float = 0.05
    margin_requirement: float = 0.5
    min_qty: float = 0.001
    freq_hours: int = 1
    fill_threshold: float = 0.05
    latency_mean: float = 1
    latency_std: float = 0.5
    sl_first: bool = True
    min_hold_bars: int = 1
    max_fill_ratio: float = 0.1
    # Static fallback spread
    spread: float = 0.0002
    # Commission tiers: (threshold_notional, maker_rate, taker_rate)
    commission_tiers: List[Tuple[float, float, float]] = field(default_factory=lambda: [
        (0,     0.0004, 0.0006),
        (10_000,0.0003, 0.0005),
        (50_000,0.0002, 0.0004),
    ])
    slippage_k: float = 0.1
    slippage_exp: float = 0.5
    spread_vol_window: int = 24
    seed: Optional[int] = None

    def __post_init__(self):
        if self.seed is not None:
            random.seed(self.seed)
        assert self.freq_hours > 0, "freq_hours must be > 0"
        assert 0 <= self.fill_threshold < 1, "fill_threshold must be in [0,1)"

    @property
    def annual_factor(self) -> float:
        periods = 365 * (24 / self.freq_hours)
        return np.sqrt(periods)

class CostModel:
    def __init__(self, cfg: BacktestConfig, spread_arr: np.ndarray):
        cfg.commission_tiers.sort(key=lambda x: x[0])
        self.cfg = cfg
        self.spread_arr = spread_arr

    def _comm_rate(self, notional: float, maker: bool) -> float:
        m0, t0 = self.cfg.commission_tiers[0][1], self.cfg.commission_tiers[0][2]
        rate = m0 if maker else t0
        for thr, m, t in self.cfg.commission_tiers:
            if notional >= thr:
                rate = m if maker else t
        return rate

    def slippage(self, qty: float, vol: float) -> float:
        if vol <= 0:
            return 0.0
        rel = min(qty / vol, 1.0)
        return self.cfg.slippage_k * (rel ** self.cfg.slippage_exp)

    def cost_pct(self, price: float, qty: float, idx: int, vol: float, maker: bool) -> float:
        spread = self.spread_arr[idx]
        notional = price * qty
        comm = self._comm_rate(notional, maker)
        slip = self.slippage(qty, vol)
        return comm + spread + slip

@dataclass
class Position:
    entry_bar: int
    entry_price: float
    qty: float
    side: int
    entry_cost: float
    is_maker: bool
    exit_bar: Optional[int] = None
    exit_price: float = 0.0
    exit_cost: float = 0.0
    pnl: float = 0.0
    slippage_notional: float = 0.0

class PositionManager:
    def __init__(self, cm: CostModel, cfg: BacktestConfig,
                 data: pd.DataFrame, ob: Optional[pd.DataFrame]):
        self.cm = cm
        self.cfg = cfg
        self.data = data
        self.ob = ob
        self.position: Optional[Position] = None
        self.trades: List[Position] = []

    def _latency(self) -> int:
        return max(0, int(round(random.gauss(self.cfg.latency_mean, self.cfg.latency_std))))

    def on_bar(self, i: int, sig: int, equity: float) -> float:
        o = self.data.open.iat[i]
        h = self.data.high.iat[i]
        l = self.data.low.iat[i]
        vol = self.data.volume.iat[i]

        # ENTRY
        if self.position is None and sig != 0:
            raw_qty = equity * self.cfg.risk_per_trade / (self.cfg.stop_loss * o)
            intended = min(raw_qty * self.cfg.leverage,
                           equity * self.cfg.leverage / o)
            max_fill = vol * self.cfg.max_fill_ratio
            qty = min(intended, max_fill)
            qty = np.floor(qty / self.cfg.min_qty) * self.cfg.min_qty

            if qty > 0:
                maker = (random.random() < 0.2)
                idx = min(i + self._latency(), len(self.data) - 1)

                # enforce top‐of‐book liquidity
                if self.ob is not None:
                    ask_size = self.ob.ask_size.iat[idx]
                    qty = min(qty, ask_size)

                fill_price = self.data.open.iat[idx] * (1 + sig/abs(sig) * self.cm.spread_arr[idx])
                cost_pct = self.cm.cost_pct(fill_price, qty, idx, vol, maker)
                self.position = Position(
                    entry_bar=idx,
                    entry_price=fill_price,
                    qty=qty,
                    side=int(np.sign(sig)),
                    entry_cost=cost_pct,
                    is_maker=maker
                )
            return 0.0

        # EXIT
        if self.position:
            hold = i - self.position.entry_bar
            if hold < self.cfg.min_hold_bars:
                return 0.0

            pos = self.position
            sl_price = pos.entry_price * (1 - pos.side * self.cfg.stop_loss)
            tp_price = pos.entry_price * (1 + pos.side * self.cfg.take_profit)

            hit_sl = (pos.side > 0 and l <= sl_price) or (pos.side < 0 and h >= sl_price)
            hit_tp = (pos.side > 0 and h >= tp_price) or (pos.side < 0 and l <= tp_price)

            first = None
            if hit_sl or hit_tp:
                first = 'sl' if hit_sl and (not hit_tp or self.cfg.sl_first) else 'tp'

            execute = False
            target = None

            if first == 'sl':
                execute, target = True, sl_price
            elif first == 'tp':
                execute, target = True, tp_price
            elif sig == 0 or np.sign(sig) != pos.side:
                execute, target = True, o

            # margin‐call exit
            if equity < pos.qty * pos.entry_price * self.cfg.margin_requirement:
                execute, target = True, (l if pos.side > 0 else h)

            if execute and target is not None:
                idx = min(i + self._latency(), len(self.data) - 1)
                if self.ob is not None:
                    bid_size = self.ob.bid_size.iat[idx]
                    pos.qty = min(pos.qty, bid_size)

                fill_pr = target * (1 - pos.side * self.cm.spread_arr[idx])
                exit_cost_pct = self.cm.cost_pct(fill_pr, pos.qty, idx, vol, False)

                # calculate PnL and true slippage notional separately
                pnl = pos.side * pos.qty * (fill_pr - pos.entry_price)
                entry_notional = pos.qty * pos.entry_price
                exit_notional = pos.qty * fill_pr
                slippage_notional = pos.entry_cost * entry_notional + exit_cost_pct * exit_notional

                pos.exit_bar = idx
                pos.exit_price = fill_pr
                pos.exit_cost = exit_cost_pct
                pos.pnl = pnl
                pos.slippage_notional = slippage_notional

                self.trades.append(pos)
                self.position = None

                return pnl

        return 0.0

class MetricsTracker:
    def __init__(self, cfg: BacktestConfig):
        self.cfg = cfg
        self.times: List[pd.Timestamp] = []
        self.eq: List[float] = []

    def record(self, t: pd.Timestamp, eq: float):
        self.times.append(t)
        self.eq.append(eq)

    def compute(self, trades: List[Position]) -> Dict[str, float]:
        eqs = pd.Series(self.eq, index=self.times)
        rtn = eqs.pct_change().dropna()
        ann = self.cfg.annual_factor

        sharpe = rtn.mean() / rtn.std() * ann if rtn.std() else np.nan
        sortino = (rtn.mean() / rtn[rtn < 0].std() * ann) if rtn[rtn < 0].std() else np.nan

        cum = (1 + rtn).cumprod()
        dd = (cum - cum.cummax()) / cum.cummax()
        mdd = dd.min()

        wins = [t.pnl for t in trades if t.pnl > 0]
        losses = [t.pnl for t in trades if t.pnl < 0]
        win_rate = len(wins) / len(trades) if trades else 0
        avg_win = np.mean(wins) if wins else 0.0
        avg_loss = np.mean(losses) if losses else 0.0
        pf = sum(wins) / abs(sum(losses)) if losses else np.nan

        var5 = np.percentile(rtn, 5) if len(rtn) else np.nan
        cvar5 = rtn[rtn <= var5].mean() if len(rtn) else np.nan
        tail_ratio = -np.percentile(rtn, 95) / var5 if var5 else np.nan

        total_return = eqs.iloc[-1] / eqs.iloc[0] - 1
        years = len(eqs) / (365 * (24 / self.cfg.freq_hours))
        cagr = (1 + total_return) ** (1 / years) - 1 if years else np.nan
        calmar = cagr / abs(mdd) if mdd else np.nan

        mean_slip = np.mean([t.slippage_notional for t in trades]) if trades else 0.0

        return {
            'Sharpe': sharpe,
            'Sortino': sortino,
            'CAGR': cagr,
            'MaxDD': mdd,
            'Calmar': calmar,
            'WinRate': win_rate,
            'AvgWin': avg_win,
            'AvgLoss': avg_loss,
            'ProfitFactor': pf,
            'VaR5%': var5,
            'CVaR5%': cvar5,
            'TailRatio': tail_ratio,
            'MeanSlippage': mean_slip,
        }

class Backtester:
    def __init__(self, data: pd.DataFrame,
                 strategy: Callable[[pd.DataFrame], pd.Series],
                 cfg: BacktestConfig,
                 ob: Optional[pd.DataFrame] = None):
        idx = pd.date_range(data.index.min(), data.index.max(), freq=f'{cfg.freq_hours}h')
        df = data.reindex(idx)
        if df['close'].isna().mean() > cfg.fill_threshold:
            raise ValueError("Missing-data fraction exceeds threshold")
        df = df.interpolate(method='time').dropna(subset=['open','high','low','close','volume'])

        if ob is not None:
            ob2 = ob.reindex(idx).interpolate().loc[df.index]
            spread_arr = ((ob2.ask - ob2.bid) / (2 * df.open)).values
        else:
            vol = df.close.pct_change().rolling(cfg.spread_vol_window).std().fillna(0)
            vol_mean = vol.mean()
            spread_arr = cfg.spread * (1 + (vol / vol_mean).values) if vol_mean>0 else np.full(len(df), cfg.spread)

        self.data, self.strategy, self.cfg = df, strategy, cfg
        self.cm = CostModel(cfg, spread_arr)
        self.pm = PositionManager(self.cm, cfg, df, ob)
        self.mt = MetricsTracker(cfg)

    def run(self) -> Dict[str, float]:
        cash = self.cfg.start_equity
        self.mt.record(self.data.index[0], cash)

        raw = self.strategy(self.data).shift(1).fillna(0)
        sig = raw.apply(lambda x: 1 if x>0 else (-1 if x<0 else 0))

        funding, leverage, last_fund = [], [], 0
        for i in range(1, len(self.data)):
            pos = self.pm.position
            unreal = (pos.side*pos.qty*(self.data.close.iat[i]-pos.entry_price)) if pos else 0.0
            eq0 = cash + unreal

            fee=0.0
            if pos and (i-last_fund)>=24:
                fee = abs(pos.qty)*self.data.close.iat[i]*0.0001
                cash-=fee; last_fund=i
            funding.append(fee)

            lev = (abs(pos.qty*pos.entry_price)/eq0) if pos else 0.0
            leverage.append(lev)

            pnl = self.pm.on_bar(i, int(sig.iat[i]), eq0)
            cash += pnl
            self.mt.record(self.data.index[i], cash)

        stats = self.mt.compute(self.pm.trades)
        # print metrics
        tbl = [[k, f"{v:.4f}"] for k,v in stats.items()]
        print(tabulate(tbl, headers=['Metric','Value'], tablefmt='fancy_grid'))

        # plotting
        fig, ax = plt.subplots(2,3,figsize=(18,10))
        eqs = pd.Series(self.mt.eq, index=self.mt.times)
        ret = eqs.pct_change().fillna(0)
        cum = (1+ret).cumprod(); dd=(cum-cum.cummax())/cum.cummax()

        ax[0,0].plot(eqs); ax[0,0].set_title('Equity Curve')
        ax[0,1].plot(dd); ax[0,1].set_title('Drawdown')
        ax[0,2].hist(ret, bins=50, density=True); ax[0,2].set_title('Returns Dist')

        w=168; ann=self.cfg.annual_factor
        ax[1,0].plot((ret.rolling(w).mean()/ret.rolling(w).std())*ann,
                     label='Sharpe');
        ax[1,0].plot(ret.rolling(w).std()*ann,label='Vol');
        ax[1,0].legend(); ax[1,0].set_title('Rolling Metrics')

        ax[1,1].fill_between(dd.index, dd.values,0);
        ax[1,1].set_title('Underwater')

        ax[1,2].plot(self.data.index[1:], funding,label='Funding');
        ax[1,2].plot(self.data.index[1:], leverage,label='Leverage');
        ax[1,2].legend(); ax[1,2].set_title('Funding & Leverage')

        plt.tight_layout(); plt.show()
        return stats

In [181]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tabulate import tabulate
from dataclasses import dataclass, field
from typing import Callable, List, Optional, Dict, Tuple
import random

@dataclass
class BacktestConfig:
    start_equity: float = 10_000.0
    risk_per_trade: float = 0.01
    leverage: float = 1.0
    stop_loss: float = 0.02
    take_profit: float = 0.05
    margin_requirement: float = 0.5
    min_qty: float = 0.001
    freq_hours: int = 1
    fill_threshold: float = 0.05
    latency_mean: float = 1
    latency_std: float = 0.5
    sl_first: bool = True
    min_hold_bars: int = 1
    max_fill_ratio: float = 0.1
    spread: float = 0.0002
    commission_tiers: List[Tuple[float, float, float]] = field(default_factory=lambda: [
        (0,     0.0004, 0.0006),
        (10_000,0.0003, 0.0005),
        (50_000,0.0002, 0.0004),
    ])
    slippage_k: float = 0.1
    slippage_exp: float = 0.5
    spread_vol_window: int = 24
    seed: Optional[int] = None

    def __post_init__(self):
        if self.seed is not None:
            random.seed(self.seed)
            np.random.seed(self.seed)
        assert self.freq_hours > 0, "freq_hours must be > 0"
        assert 0 <= self.fill_threshold < 1, "fill_threshold must be in [0,1)"

    @property
    def annual_factor(self) -> float:
        periods = 365 * (24 / self.freq_hours)
        return np.sqrt(periods)

class CostModel:
    def __init__(self, cfg: BacktestConfig, spread_arr: np.ndarray):
        cfg.commission_tiers.sort(key=lambda x: x[0])
        self.cfg = cfg
        self.spread_arr = spread_arr

    def _comm_rate(self, notional: float, maker: bool) -> float:
        rate = self.cfg.commission_tiers[0][1 if maker else 2]
        for thr, m, t in self.cfg.commission_tiers:
            if notional >= thr:
                rate = m if maker else t
        return rate

    def slippage(self, qty: float, vol: float) -> float:
        if vol <= 0:
            return 0.0
        rel = min(qty / vol, 1.0)
        return self.cfg.slippage_k * (rel ** self.cfg.slippage_exp)

    def cost_pct(self, price: float, qty: float, idx: int, vol: float, maker: bool) -> float:
        # cost excludes spread, which is realized in fill price
        notional = price * qty
        comm = self._comm_rate(notional, maker)
        slip = self.slippage(qty, vol)
        return comm + slip

@dataclass
class Position:
    entry_bar: int
    entry_price: float
    qty: float
    side: int
    entry_cost: float
    is_maker: bool
    exit_bar: Optional[int] = None
    exit_price: float = 0.0
    exit_cost: float = 0.0
    pnl: float = 0.0
    slippage_notional: float = 0.0

class PositionManager:
    def __init__(self, cm: CostModel, cfg: BacktestConfig,
                 data: pd.DataFrame, ob: Optional[pd.DataFrame]):
        self.cm = cm
        self.cfg = cfg
        self.data = data
        self.ob = ob
        self.position: Optional[Position] = None
        self.trades: List[Position] = []

    def _latency(self) -> int:
        return max(0, int(round(random.gauss(self.cfg.latency_mean, self.cfg.latency_std))))

    def on_bar(self, i: int, sig: int, equity: float) -> float:
        o, h, l, vol = (self.data[col].iat[i] for col in ['open','high','low','volume'])

        # ENTRY
        if self.position is None and sig != 0:
            raw_qty = equity * self.cfg.risk_per_trade / (self.cfg.stop_loss * o)
            intended = min(raw_qty * self.cfg.leverage,
                           equity * self.cfg.leverage / o)
            max_fill = vol * self.cfg.max_fill_ratio
            qty = min(intended, max_fill)
            qty = np.floor(qty / self.cfg.min_qty) * self.cfg.min_qty

            if qty > 0:
                maker = (random.random() < 0.2)
                idx = min(i + self._latency(), len(self.data) - 1)
                if self.ob is not None:
                    ask_size = self.ob.ask_size.iat[idx]
                    qty = min(qty, ask_size)

                spread = self.cm.spread_arr[idx]
                fill_price = self.data.open.iat[idx] * (1 + sig/abs(sig) * spread)
                cost_pct = self.cm.cost_pct(fill_price, qty, idx, vol, maker)
                self.position = Position(
                    entry_bar=idx,
                    entry_price=fill_price,
                    qty=qty,
                    side=int(np.sign(sig)),
                    entry_cost=cost_pct,
                    is_maker=maker
                )
            return 0.0

        # EXIT logic unchanged aside from cost_pct behavior
        if self.position:
            hold = i - self.position.entry_bar
            if hold < self.cfg.min_hold_bars:
                return 0.0

            pos = self.position
            sl_price = pos.entry_price * (1 - pos.side * self.cfg.stop_loss)
            tp_price = pos.entry_price * (1 + pos.side * self.cfg.take_profit)

            hit_sl = (pos.side > 0 and l <= sl_price) or (pos.side < 0 and h >= sl_price)
            hit_tp = (pos.side > 0 and h >= tp_price) or (pos.side < 0 and l <= tp_price)
            first = None
            if hit_sl or hit_tp:
                first = 'sl' if hit_sl and (not hit_tp or self.cfg.sl_first) else 'tp'

            execute = False
            target = None
            if first == 'sl':
                execute, target = True, sl_price
            elif first == 'tp':
                execute, target = True, tp_price
            elif sig == 0 or np.sign(sig) != pos.side:
                execute, target = True, o

            if equity < pos.qty * pos.entry_price * self.cfg.margin_requirement:
                execute, target = True, (l if pos.side > 0 else h)

            if execute and target is not None:
                idx = min(i + self._latency(), len(self.data) - 1)
                if self.ob is not None:
                    bid_size = self.ob.bid_size.iat[idx]
                    pos.qty = min(pos.qty, bid_size)

                fill_pr = target * (1 - pos.side * self.cm.spread_arr[idx])
                exit_cost_pct = self.cm.cost_pct(fill_pr, pos.qty, idx, vol, False)

                pnl = pos.side * pos.qty * (fill_pr - pos.entry_price)
                entry_notional = pos.qty * pos.entry_price
                exit_notional = pos.qty * fill_pr
                slippage_notional = pos.entry_cost * entry_notional + exit_cost_pct * exit_notional

                pos.exit_bar = idx
                pos.exit_price = fill_pr
                pos.exit_cost = exit_cost_pct
                pos.pnl = pnl
                pos.slippage_notional = slippage_notional

                self.trades.append(pos)
                self.position = None
                return pnl

        return 0.0

class MetricsTracker:
    def __init__(self, cfg: BacktestConfig):
        self.cfg = cfg
        self.times: List[pd.Timestamp] = []
        self.eq: List[float] = []

    def record(self, t: pd.Timestamp, eq: float):
        self.times.append(t)
        self.eq.append(eq)

    def compute(self, trades: List[Position]) -> Dict[str, float]:
        eqs = pd.Series(self.eq, index=self.times)
        rtn = eqs.pct_change().dropna()
        ann = self.cfg.annual_factor

        sharpe = rtn.mean() / rtn.std() * ann if rtn.std() else np.nan
        sortino = rtn.mean() / rtn[rtn < 0].std() * ann if rtn[rtn < 0].std() else np.nan

        cum = (1 + rtn).cumprod()
        dd = (cum - cum.cummax()) / cum.cummax()
        mdd = dd.min()

        wins = [t.pnl for t in trades if t.pnl > 0]
        losses = [t.pnl for t in trades if t.pnl < 0]
        win_rate = len(wins) / len(trades) if trades else 0
        avg_win = np.mean(wins) if wins else 0.0
        avg_loss = np.mean(losses) if losses else 0.0
        pf = sum(wins) / abs(sum(losses)) if losses else np.nan

        var5 = np.percentile(rtn, 5) if len(rtn) else np.nan
        cvar5 = rtn[rtn <= var5].mean() if len(rtn) else np.nan
        tail_ratio = -np.percentile(rtn, 95) / var5 if var5 else np.nan

        total_return = eqs.iloc[-1] / eqs.iloc[0] - 1
        years = len(eqs) / (365 * (24 / self.cfg.freq_hours))
        cagr = (1 + total_return) ** (1 / years) - 1 if years else np.nan
        calmar = cagr / abs(mdd) if mdd else np.nan

        mean_slip = np.mean([t.slippage_notional for t in trades]) if trades else 0.0

        return {
            'Sharpe': sharpe, 'Sortino': sortino, 'CAGR': cagr,
            'MaxDD': mdd, 'Calmar': calmar, 'WinRate': win_rate,
            'AvgWin': avg_win, 'AvgLoss': avg_loss,
            'ProfitFactor': pf, 'VaR5%': var5, 'CVaR5%': cvar5,
            'TailRatio': tail_ratio, 'MeanSlippage': mean_slip
        }

class Backtester:
    def __init__(self, data: pd.DataFrame,
                 strategy: Callable[[pd.DataFrame], pd.Series],
                 cfg: BacktestConfig,
                 ob: Optional[pd.DataFrame] = None):
        # build fixed-frequency bars via resampling
        df = data.resample(f'{cfg.freq_hours}H').agg({
            'open':'first', 'high':'max', 'low':'min',
            'close':'last', 'volume':'sum'
        }).dropna(subset=['open','high','low','close','volume'])

        # missing-data check on close only
        missing = df['close'].isna().sum()
        if missing / len(df) > cfg.fill_threshold:
            raise ValueError("Missing-data fraction exceeds threshold")

        # dynamic spread
        if ob is not None:
            ob2 = ob.resample(f'{cfg.freq_hours}H').agg({
                'bid':'last','ask':'last','bid_size':'sum','ask_size':'sum'
            }).reindex(df.index)
            spread_arr = ((ob2.ask - ob2.bid) / (2 * df.open)).values
        else:
            vol = df.close.pct_change().rolling(cfg.spread_vol_window).std().fillna(0)
            vol_mean = vol.mean()
            spread_arr = cfg.spread * (1 + (vol / vol_mean).values) if vol_mean>0 else np.full(len(df), cfg.spread)

        self.data = df
        self.strategy = strategy
        self.cfg = cfg
        self.cm = CostModel(cfg, spread_arr)
        self.pm = PositionManager(self.cm, cfg, df, ob)
        self.mt = MetricsTracker(cfg)

    def run(self) -> Dict[str, float]:
        cash = self.cfg.start_equity
        self.mt.record(self.data.index[0], cash)

        raw_sig = self.strategy(self.data).shift(1).fillna(0)
        sig = raw_sig.apply(lambda x: 1 if x>0 else (-1 if x<0 else 0))

        funding, leverage = [], []
        last_fund = 0

        for i in range(1, len(self.data)):
            pos = self.pm.position
            unreal = pos.side * pos.qty * (self.data.close.iat[i] - pos.entry_price) if pos else 0.0
            eq0 = cash + unreal

            fee = 0.0
            if pos and (i - last_fund) >= 24:
                fee = abs(pos.qty) * self.data.close.iat[i] * 0.0001
                cash -= fee
                last_fund = i
            funding.append(fee)

            lev = (abs(pos.qty * pos.entry_price) / eq0) if pos else 0.0
            leverage.append(lev)

            pnl = self.pm.on_bar(i, int(sig.iat[i]), eq0)
            cash += pnl
            self.mt.record(self.data.index[i], cash)

        stats = self.mt.compute(self.pm.trades)
        table = [[k, f"{v:.4f}"] for k, v in stats.items()]
        print(tabulate(table, headers=['Metric','Value'], tablefmt='fancy_grid'))

        # Generate plots (omitted for brevity)
        return stats


In [142]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tabulate import tabulate
from dataclasses import dataclass, field
from typing import Callable, List, Optional, Dict, Tuple
import random

@dataclass
class BacktestConfig:
    start_equity: float = 10_000.0
    risk_per_trade: float = 0.01
    leverage: float = 1.0
    stop_loss: float = 0.02
    take_profit: float = 0.05
    initial_margin: float = 0.5
    maintenance_margin: float = 0.25
    min_qty: float = 0.001
    freq_hours: int = 1
    fill_threshold: float = 0.05
    latency_mean_sec: float = 300  # latency in seconds
    latency_std_sec: float = 100
    sl_first: bool = True
    min_hold_bars: int = 1
    spread: float = 0.0002
    commission_tiers: List[Tuple[float, float, float]] = field(default_factory=lambda: [
        (0,     0.0004, 0.0006),
        (10_000,0.0003, 0.0005),
        (50_000,0.0002, 0.0004),
    ])
    slippage_coeff: float = 0.1
    slippage_exp: float = 0.5
    impact_coeff: float = 0.1
    impact_exp: float = 0.5
    funding_rates: Optional[pd.Series] = None  # hourly funding rate series aligned with data
    seed: Optional[int] = None

    def __post_init__(self):
        if self.seed is not None:
            random.seed(self.seed)
            np.random.seed(self.seed)
        assert 0 < self.freq_hours <= 24
        assert 0 <= self.fill_threshold < 1
        assert 0 < self.maintenance_margin < self.initial_margin

    @property
    def annual_factor(self) -> float:
        periods = 365 * (24 / self.freq_hours)
        return np.sqrt(periods)

class CostModel:
    def __init__(self, cfg: BacktestConfig, spread_arr: np.ndarray, volume: np.ndarray):
        cfg.commission_tiers.sort(key=lambda x: x[0])
        self.cfg = cfg
        self.spread_arr = spread_arr
        self.volume = volume

    def commission_rate(self, notional: float, maker: bool) -> float:
        rate = 0.0
        for thr, m, t in self.cfg.commission_tiers:
            if notional >= thr:
                rate = m if maker else t
        return rate

    def slippage_pct(self, qty: float, vol: float) -> float:
        if vol <= 0:
            return 0.0
        rel = min(qty / vol, 1.0)
        return self.cfg.slippage_coeff * (rel ** self.cfg.slippage_exp)

    def impact_pct(self, qty: float, vol: float) -> float:
        if vol <= 0:
            return 0.0
        rel = qty / vol
        return self.cfg.impact_coeff * (rel ** self.cfg.impact_exp)

    def cost_pct(self, price: float, qty: float, idx: int, maker: bool) -> float:
        notional = price * qty
        comm = self.commission_rate(notional, maker)
        slip = self.slippage_pct(qty, self.volume[idx])
        return comm + slip

    def adjust_price(self, price: float, qty: float, idx: int, side: int) -> float:
        sp = self.spread_arr[idx]
        imp = self.impact_pct(qty, self.volume[idx])
        return price * (1 + side * (sp + imp))

@dataclass
class Position:
    entry_bar: int
    entry_price: float
    qty: float
    side: int
    entry_cost: float
    is_maker: bool
    exit_bar: Optional[int] = None
    exit_price: float = 0.0
    exit_cost: float = 0.0
    pnl: float = 0.0
    slippage_notional: float = 0.0

class PositionManager:
    def __init__(self, cm: CostModel, cfg: BacktestConfig,
                 data: pd.DataFrame, mark_price: Optional[pd.Series]):
        self.cm = cm
        self.cfg = cfg
        self.data = data
        self.mark_price = mark_price
        self.position: Optional[Position] = None
        self.trades: List[Position] = []

    def _latency_bars(self) -> float:
        bar_secs = self.cfg.freq_hours * 3600
        lat = max(0, random.gauss(self.cfg.latency_mean_sec, self.cfg.latency_std_sec))
        return lat / bar_secs

    def on_bar(self, i: int, sig: int, equity: float) -> float:
        o, h, l = (self.data[col].iat[i] for col in ['open','high','low'])
        vol = self.data.volume.iat[i]

        if self.position is None and sig != 0:
            raw_qty = equity * self.cfg.risk_per_trade / (self.cfg.stop_loss * o)
            qty = min(raw_qty * self.cfg.leverage, equity * self.cfg.leverage / o)
            qty = min(qty, vol * self.cfg.fill_threshold)
            qty = np.floor(qty / self.cfg.min_qty) * self.cfg.min_qty
            if qty <= 0:
                return 0.0

            maker = (random.random() < 0.2)
            idx = min(i + int(np.ceil(self._latency_bars())), len(self.data)-1)
            price0 = self.data.open.iat[idx]
            fill_price = self.cm.adjust_price(price0, qty, idx, sig)
            cost_pct = self.cm.cost_pct(fill_price, qty, idx, maker)
            self.position = Position(i, fill_price, qty, sig, cost_pct, maker)
            return 0.0

        if self.position:
            pos = self.position
            hold = i - pos.entry_bar
            if hold < self.cfg.min_hold_bars:
                return 0.0

            mark = self.mark_price.iat[i] if self.mark_price is not None else self.data.close.iat[i]
            if (pos.side > 0 and mark <= pos.entry_price * (1 - self.cfg.maintenance_margin)) or \
               (pos.side < 0 and mark >= pos.entry_price * (1 + self.cfg.maintenance_margin)):
                execute, target, liq_fee = True, mark, 0.005
            else:
                slp = pos.entry_price * (1 - pos.side * self.cfg.stop_loss)
                tpp = pos.entry_price * (1 + pos.side * self.cfg.take_profit)
                hit_sl = (pos.side>0 and l<=slp) or (pos.side<0 and h>=slp)
                hit_tp = (pos.side>0 and h>=tpp) or (pos.side<0 and l<=tpp)
                first = None
                if hit_sl or hit_tp:
                    first = 'sl' if hit_sl and (not hit_tp or self.cfg.sl_first) else 'tp'
                if first == 'sl': execute, target, liq_fee = True, slp, 0
                elif first == 'tp': execute, target, liq_fee = True, tpp, 0
                elif sig == 0 or np.sign(sig) != pos.side: execute, target, liq_fee = True, o, 0
                else: execute = False

            if execute:
                idx = min(i + int(np.ceil(self._latency_bars())), len(self.data)-1)
                price0 = self.data.open.iat[idx]
                fill_price = self.cm.adjust_price(target, pos.qty, idx, -pos.side)
                exit_cost_pct = self.cm.cost_pct(fill_price, pos.qty, idx, False)
                pnl = pos.side * pos.qty * (fill_price - pos.entry_price)
                pos.exit_bar = idx
                pos.exit_price = fill_price
                pos.exit_cost = exit_cost_pct + liq_fee
                pos.pnl = pnl - liq_fee * pos.qty * pos.entry_price
                self.trades.append(pos)
                self.position = None
                return pos.pnl
        return 0.0

class MetricsTracker:
    def __init__(self, cfg: BacktestConfig):
        self.cfg = cfg
        self.times, self.eq = [], []
    def record(self, t, eq):
        self.times.append(t)
        self.eq.append(eq)
    def compute(self, trades: List[Position]) -> Dict[str, float]:
        eqs = pd.Series(self.eq, index=self.times)
        rtn = eqs.pct_change().dropna()
        ann = self.cfg.annual_factor
        cum = (1 + rtn).cumprod()
        dd = (cum - cum.cummax()) / cum.cummax()
        maxdd = dd.min()
        mean_slip = np.mean([t.slippage_notional for t in trades]) if trades else 0.0
        stats = {
            'Sharpe': rtn.mean()/rtn.std()*ann if rtn.std() else np.nan,
            'CAGR': (eqs.iloc[-1]/eqs.iloc[0])**(1/(len(eqs)/(365*(24/self.cfg.freq_hours))))-1,
            'MaxDD': maxdd,
            'MeanSlippage': mean_slip
        }
        return stats

class Backtester:
    def __init__(self, data: pd.DataFrame,
                 strategy: Callable[[pd.DataFrame], pd.Series],
                 cfg: BacktestConfig,
                 ob: Optional[pd.DataFrame] = None,
                 mark_price: Optional[pd.Series] = None):
        # fixed-frequency aggregation
        df = data.resample(f'{cfg.freq_hours}h').agg({'open':'first', 'high':'max', 'low':'min', 'close':'last', 'volume':'sum'}).dropna()
        fund = cfg.funding_rates.reindex(df.index) if cfg.funding_rates is not None else pd.Series(0, index=df.index)
        if ob is not None:
            ob2 = ob.resample(f'{cfg.freq_hours}h').agg({'bid':'last','ask':'last'}).reindex(df.index)
            spread_arr = ((ob2['ask'] - ob2['bid'])/(2*df['open'])).values
        else:
            vol = df['close'].pct_change().rolling(24).std().fillna(0)
            spread_arr = (cfg.spread * (1 + vol/vol.mean())).values

        self.data, self.cfg = df, cfg
        self.cm = CostModel(cfg, spread_arr, df['volume'].values)
        self.pm = PositionManager(self.cm, cfg, df, mark_price)
        self.mt = MetricsTracker(cfg)
        self.fund_rates = fund
        self.strategy = strategy

    def run(self) -> Dict[str, float]:
        cash = self.cfg.start_equity
        self.mt.record(self.data.index[0], cash)
        sig = self.strategy(self.data).shift(1).fillna(0).clip(-1,1)
        for i in range(1, len(self.data)):
            unreal = self.pm.position.side * self.pm.position.qty * (self.data['close'].iat[i]-self.pm.position.entry_price) if self.pm.position else 0
            eq0 = cash + unreal
            fr = self.fund_rates.iat[i]
            cash -= abs(self.pm.position.qty)*self.data['close'].iat[i]*fr if self.pm.position else 0
            pnl = self.pm.on_bar(i, int(sig.iat[i]), eq0)
            cash += pnl
            self.mt.record(self.data.index[i], cash)
        stats = self.mt.compute(self.pm.trades)
        print(tabulate([[k,f"{v:.4f}"] for k,v in stats.items()], headers=['Metric','Value'], tablefmt='fancy_grid'))
        return stats


In [144]:
import pandas as pd

# Load without parsing dates
df = pd.read_csv("ohlcv.csv")

# Convert UNIX timestamp (ms) to datetime
df["start_time"] = pd.to_datetime(df["start_time"], unit="ms")

# Set index to datetime
df.set_index("start_time", inplace=True)

# Sort index for consistency
df = df.sort_index()

# Make column names lowercase
df.columns = df.columns.str.lower()

# Now run the backtest
cfg = BacktestConfig()

# Simple SMA momentum
sma_strat = SimpleMAMomentum(window=24)
bt1 = Backtester(df, sma_strat.generate_signals, cfg)
res1 = bt1.run()

# Enhanced MACD+RSI strategy
macd_strat = EnhancedMACDRSIStrategy()
bt2 = Backtester(df, macd_strat.generate_signals, cfg)
res2 = bt2.run()

╒══════════════╤═════════╕
│ Metric       │   Value │
╞══════════════╪═════════╡
│ Sharpe       │ -4.2792 │
├──────────────┼─────────┤
│ CAGR         │ -0.6534 │
├──────────────┼─────────┤
│ MaxDD        │ -0.6586 │
├──────────────┼─────────┤
│ MeanSlippage │  0      │
╘══════════════╧═════════╛
╒══════════════╤═════════╕
│ Metric       │   Value │
╞══════════════╪═════════╡
│ Sharpe       │ -2.582  │
├──────────────┼─────────┤
│ CAGR         │ -0.0086 │
├──────────────┼─────────┤
│ MaxDD        │ -0.0086 │
├──────────────┼─────────┤
│ MeanSlippage │  0      │
╘══════════════╧═════════╛


In [168]:
import numpy as np
import pandas as pd
from tabulate import tabulate
from dataclasses import dataclass
from typing import Callable, List, Optional, Dict
import matplotlib.pyplot as plt

@dataclass
class BacktestConfig:
    start_equity: float = 10_000.0
    risk_per_trade: float = 0.01
    leverage: float = 1.0
    stop_loss: float = 0.02
    take_profit: float = 0.05
    entry_margin_ratio: float = 0.5
    maintenance_margin_ratio: float = 0.25
    min_qty: float = 0.001
    min_hold_bars: int = 1
    freq_hours: int = 1
    fill_threshold: float = 0.05
    spread: float = 0.0002
    cost_tiers: Optional[Dict[float, float]] = None

    def __post_init__(self):
        assert 0 < self.freq_hours <= 24, "freq_hours must be between 1 and 24"
        assert 0 <= self.fill_threshold < 1, "fill_threshold must be in [0,1)"
        assert 0 < self.maintenance_margin_ratio < self.entry_margin_ratio <= 1.0, \
               "Require 0 < maintenance_margin_ratio < entry_margin_ratio <= 1"

class CostModel:
    def __init__(self, spread: float, tiers: Optional[Dict[float, float]] = None):
        self.base_spread = spread
        self.tiers = tiers or {}

    def cost_pct(self, price: float, qty: float) -> float:
        notional = price * qty
        for threshold, pct in sorted(self.tiers.items(), key=lambda x: x[0], reverse=True):
            if notional >= threshold:
                return pct
        return self.base_spread

    def adjust_price(self, price: float, side: int, qty: float) -> float:
        pct = self.cost_pct(price, qty)
        return price * (1 + side * pct)

@dataclass
class Position:
    entry_bar: int
    entry_price: float
    qty: float
    side: int
    entry_cost: float
    locked_margin: float
    exit_bar: Optional[int] = None
    exit_price: float = 0.0
    exit_cost: float = 0.0
    pnl: float = 0.0
    exit_reason: Optional[str] = None

class PositionManager:
    def __init__(self, cfg: BacktestConfig, cost_model: CostModel, data: pd.DataFrame):
        self.cfg = cfg
        self.cm = cost_model
        self.data = data
        self.position: Optional[Position] = None
        self.trades: List[Position] = []
        self.reserved_cash: float = 0.0
        self._last_entry_cost: float = 0.0

    def _enter(self, i: int, sig: int, equity: float) -> None:
        o = self.data['open'].iat[i]
        available = equity - self.reserved_cash
        raw_qty = available * self.cfg.risk_per_trade / (self.cfg.stop_loss * o)
        qty = min(raw_qty * self.cfg.leverage, available * self.cfg.leverage / o)
        qty = min(qty, self.data['volume'].iat[i] * self.cfg.fill_threshold)
        qty = np.floor(qty / self.cfg.min_qty) * self.cfg.min_qty
        if qty <= 0:
            return
        price = self.cm.adjust_price(o, sig, qty)
        cost_pct = self.cm.cost_pct(price, qty)
        locked = qty * price * self.cfg.entry_margin_ratio
        entry_fee = cost_pct * qty * price
        self.reserved_cash += locked
        self.position = Position(i, price, qty, sig, cost_pct, locked)
        self._last_entry_cost = locked + entry_fee

    def _exit(self, pos: Position, i: int, fill_price: float, cost_pct: float, reason: str) -> None:
        entry_fee = pos.entry_cost * pos.qty * pos.entry_price
        exit_fee = cost_pct * pos.qty * fill_price
        pnl = pos.side * pos.qty * (fill_price - pos.entry_price) - entry_fee - exit_fee
        self.reserved_cash -= pos.locked_margin
        pos.exit_bar = i
        pos.exit_price = fill_price
        pos.exit_cost = cost_pct
        pos.pnl = pnl
        pos.exit_reason = reason
        self.trades.append(pos)

    def on_bar(self, i: int, sig: int, equity: float) -> float:
        self._last_entry_cost = 0.0  # reset carryover
        cash_change = 0.0
        if self.position:
            # margin call logic
            unreal = self.position.side * self.position.qty * (
                self.data['close'].iat[i] - self.position.entry_price)
            available = equity - self.reserved_cash
            maintenance_req = self.position.qty * self.position.entry_price * self.cfg.maintenance_margin_ratio
            if available < maintenance_req:
                fill = self.cm.adjust_price(self.data['close'].iat[i], -self.position.side, self.position.qty)
                cost_pct = self.cm.cost_pct(fill, self.position.qty)
                self._exit(self.position, i, fill, cost_pct, 'MarginCall')
                cash_change += self.trades[-1].locked_margin + self.trades[-1].pnl
                self.position = None
                return cash_change
        # entry
        if self.position is None:
            if sig != 0:
                self._enter(i, sig, equity)
                cash_change -= self._last_entry_cost
            return cash_change
        # enforce minimum hold
        if i - self.position.entry_bar < self.cfg.min_hold_bars:
            return cash_change
        # Intra-bar ordering: decide between TP and SL by whichever condition first in code
        pos = self.position
        o, h, l = (self.data[col].iat[i] for col in ['open', 'high', 'low'])
        slp = pos.entry_price * (1 - pos.side * self.cfg.stop_loss)
        tpp = pos.entry_price * (1 + pos.side * self.cfg.take_profit)
        # TP first, then SL, then signal
        if (pos.side > 0 and h >= tpp) or (pos.side < 0 and l <= tpp):
            reason, target = 'TP', tpp
        elif (pos.side > 0 and l <= slp) or (pos.side < 0 and h >= slp):
            reason, target = 'SL', slp
        elif sig == 0 or np.sign(sig) != pos.side:
            reason, target = 'Signal', o
        else:
            return cash_change
        fill = self.cm.adjust_price(target, -pos.side, pos.qty)
        cost_pct = self.cm.cost_pct(fill, pos.qty)
        self._exit(pos, i, fill, cost_pct, reason)
        cash_change += pos.locked_margin + pos.pnl
        self.position = None
        return cash_change

class MetricsTracker:
    def __init__(self, cfg: BacktestConfig):
        self.cfg = cfg
        self.times: List[pd.Timestamp] = []
        self.eq: List[float] = []

    def record(self, time: pd.Timestamp, equity: float):
        self.times.append(time)
        self.eq.append(equity)

    def compute(self, trades: List[Position]) -> Dict[str, float]:
        eqs = pd.Series(self.eq, index=self.times)
        rtn = eqs.pct_change().dropna()
        ann = np.sqrt(365 * 24 / self.cfg.freq_hours)
        total_return = eqs.iloc[-1] / eqs.iloc[0] - 1
        cagr = (1 + total_return) ** (1/(len(eqs) / (365*24/self.cfg.freq_hours))) - 1
        vol = rtn.std() * ann
        sharpe = rtn.mean() / rtn.std() * ann
        sortino = rtn.mean() / rtn[rtn < 0].std() * ann
        cum = (1 + rtn).cumprod()
        dd = cum / cum.cummax() - 1
        max_dd = dd.min()
        calmar = cagr / abs(max_dd)
        wins = [t.pnl for t in trades if t.pnl > 0]
        losses = [t.pnl for t in trades if t.pnl < 0]
        stats = {
            'TotalReturn': total_return, 'CAGR': cagr,
            'Sharpe': sharpe, 'Sortino': sortino,
            'Volatility': vol, 'MaxDrawdown': max_dd,
            'Calmar': calmar,
            'WinRate': len(wins)/len(trades) if trades else np.nan,
            'AvgWin': np.mean(wins) if wins else np.nan,
            'AvgLoss': np.mean(losses) if losses else np.nan,
            'ProfitFactor': sum(wins)/abs(sum(losses)) if losses else np.nan
        }
        reasons = pd.Series([t.exit_reason for t in trades])
        for r in ['SL', 'TP', 'Signal', 'Forced', 'MarginCall']:
            stats[f'{r}_Exits'] = (reasons == r).sum()
        return stats

class Backtester:
    def __init__(self, data: pd.DataFrame, strategy: Callable, cfg: BacktestConfig):
        self.df = data.resample(f'{cfg.freq_hours}h').agg(
            {'open': 'first', 'high': 'max', 'low': 'min', 'close': 'last', 'volume': 'sum'}
        ).dropna()
        self.cfg = cfg
        cm = CostModel(cfg.spread, cfg.cost_tiers)
        self.pm = PositionManager(cfg, cm, self.df)
        self.mt = MetricsTracker(cfg)
        self.strategy = strategy
        self.equity_curve: List[float] = []

    def _simulate(self):
        cash = self.cfg.start_equity
        self.equity_curve = [cash]
        self.mt.times, self.mt.eq = [], []
        self.mt.record(self.df.index[0], cash)
        sig = self.strategy(self.df).shift(1).fillna(0).clip(-1, 1)
        for i in range(1, len(self.df)):
            unreal = self.position_unreal(i) if self.pm.position else 0.0
            equity_pre = cash + unreal
            delta = self.pm.on_bar(i, int(sig.iat[i]), equity_pre)
            cash += delta
            unreal2 = self.position_unreal(i) if self.pm.position else 0.0
            equity_post = cash + unreal2
            self.equity_curve.append(equity_post)
            self.mt.record(self.df.index[i], equity_post)

    def position_unreal(self, i: int) -> float:
        return self.pm.position.side * self.pm.position.qty * (
            self.df['close'].iat[i] - self.pm.position.entry_price
        )

    def _force_close(self):
        if self.pm.position:
            i = len(self.df) - 1
            pos = self.pm.position
            fill = self.pm.cm.adjust_price(self.df['close'].iat[i], -pos.side, pos.qty)
            cost_pct = self.pm.cm.cost_pct(fill, pos.qty)
            entry_fee = pos.entry_cost * pos.qty * pos.entry_price
            exit_fee = cost_pct * pos.qty * fill
            pnl = pos.side * pos.qty * (fill - pos.entry_price) - entry_fee - exit_fee
            self.pm.reserved_cash -= pos.locked_margin
            self.equity_curve[-1] += pos.locked_margin + pnl
            self.mt.eq[-1] = self.equity_curve[-1]
            pos.exit_bar, pos.exit_price, pos.exit_cost, pos.pnl, pos.exit_reason = (
                i, fill, cost_pct, pnl, 'Forced'
            )
            self.pm.trades.append(pos)
            self.pm.position = None

    def run(self) -> Dict[str, float]:
        self._simulate()
        self._force_close()
        stats = self.mt.compute(self.pm.trades)
        print(tabulate([[k, f"{v:.4f}"] for k, v in stats.items()], headers=['Metric', 'Value'], tablefmt='fancy_grid'))
        return stats


In [170]:
import pandas as pd

# Load without parsing dates
df = pd.read_csv("ohlcv.csv")

# Convert UNIX timestamp (ms) to datetime
df["start_time"] = pd.to_datetime(df["start_time"], unit="ms")

# Set index to datetime
df.set_index("start_time", inplace=True)

# Sort index for consistency
df = df.sort_index()

# Make column names lowercase
df.columns = df.columns.str.lower()

# Now run the backtest
cfg = BacktestConfig()

# Simple SMA momentum
sma_strat = SimpleMAMomentum(window=24)
bt1 = Backtester(df, sma_strat.generate_signals, cfg)
res1 = bt1.run()

# Enhanced MACD+RSI strategy
macd_strat = EnhancedMACDRSIStrategy()
bt2 = Backtester(df, macd_strat.generate_signals, cfg)
res2 = bt2.run()

╒══════════════════╤══════════╕
│ Metric           │    Value │
╞══════════════════╪══════════╡
│ TotalReturn      │  -0.3661 │
├──────────────────┼──────────┤
│ CAGR             │  -0.3653 │
├──────────────────┼──────────┤
│ Sharpe           │   5.8337 │
├──────────────────┼──────────┤
│ Sortino          │   7.7988 │
├──────────────────┼──────────┤
│ Volatility       │  12.1673 │
├──────────────────┼──────────┤
│ MaxDrawdown      │  -0.5339 │
├──────────────────┼──────────┤
│ Calmar           │  -0.6842 │
├──────────────────┼──────────┤
│ WinRate          │   0.2429 │
├──────────────────┼──────────┤
│ AvgWin           │  88.6623 │
├──────────────────┼──────────┤
│ AvgLoss          │ -32.8996 │
├──────────────────┼──────────┤
│ ProfitFactor     │   0.8645 │
├──────────────────┼──────────┤
│ SL_Exits         │ 119      │
├──────────────────┼──────────┤
│ TP_Exits         │  69      │
├──────────────────┼──────────┤
│ Signal_Exits     │ 689      │
├──────────────────┼──────────┤
│ Forced